<div class="alert alert-info">
<H2> Capstone 1 Predictive Maintenance </H2>
<H3> Modeling </H3>
    
Steps involved:
    <ol>
        <li>___Import libraries and data___</li>
        <li>___Select modeling technique___: As the second step in this phase of the project: modeling, I will select the actual modeling technique that I will be applying. </li>
        <li>___Generate test design___: Before I actually build a model, I need to generate the procedure or mechanism to test the model’s validity and quality. This process may include for example, in supervised data mining tasks such as classification, it is common to use error rates as a quality measure for modeling. Accordingly, I will split the dataset into 1 — a training set, and 2.  a testing set, while building the model on the training set, and estimating its quality on the separate testing set.</li>
        <li>___Build model___: Here I will run the modeling tool on the prepared dataset to create one or more models.</li>
        <li>___Assess the model___: Here I will interpret the models according to my domain knowledge, the data mining success criteria, and the anticipated test design. I will also decide on the success of the application of the modeling and discovery methods technically, may contact with concerned business analysts and domain experts later to discuss data mining results in the appropriate business perspective. While this task considers models, the evaluation phase (Phase 5) further takes into account all other effects that occurred in the course of this analysis.
At this stage, it is also important to rank the constructed models while assessing them according to any evaluation criteria. I will take the business objectives' success criteria into account as far as possible here.
</li>
    </ol>
    


<div class="alert alert-success">
<H3> Step 1. Import libraries and data </H3>

In [1]:
# imports
import pandas as pd
import numpy as np
from IPython.core.display import HTML

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# statistical processing
from xgboost import XGBClassifier, cv, plot_importance
from sklearn import preprocessing, scale
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

# data viz imports
import seaborn as sns
import matplotlib.pyplot as plt

# viz styling
sns.set_style("whitegrid")
sns.set_context("poster")
plt.style.use('ggplot')
%matplotlib inline

# for reading csv files
import os
# for reading csv files
import glob 
# for trig on angles
import math 
# to search strings for nums
import re
# to combine final dfs into a single df
from functools import reduce 

# load data
df1 = pd.read_csv('data/sensor_cleaned.csv')

ImportError: cannot import name 'scale' from 'sklearn' (C:\Users\alfre\Anaconda3\lib\site-packages\sklearn\__init__.py)

<div class="alert alert-success">
<H3> Step 2. Select modeling technique </H3>
    
Sub-Steps involved:
    <ol>
        <li>___Modeling technique___: I have document the actual modeling technique utilized. </li>
        <li>___Modelling assumptions___: Many modelling techniques make specific assumptions about the data, for example that all attributes have uniform distributions, no missing values allowed, class attribute must be symbolic etc. Record any assumptions made.</li>
    </ol>

<div class="alert alert-success">
<H3> Step 3. Generate test design </H3>
    
Sub-Steps involved:
    <ol>
        <li>Here I will describe the intended plan for training, testing, and evaluating the select modeling effort. A chief component of the process here is determining how to divide the available dataset into training, test , and validation (Hold-Out) datasets. </li>
    </ol>

In [22]:
# get first five records
df1.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.98958,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,0
1,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.98958,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,0
2,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,NaN,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.46875,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,0
3,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,NaN,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.46875,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,0
4,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,NaN,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.98958,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,0


In [27]:
# create x-variables and y(target) variable
X = df1.drop('machine_status', axis=1).values 
y = df1['machine_status'].values

# setting zero values to mean
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
logreg = LogisticRegression(solver='liblinear')
steps = [('imputation', imp), ('logistic_regression', logreg)]
pipeline = Pipeline(steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [12]:
# reshaping the array
y = y.reshape(-1, 1) 
X = X.reshape(-1, 1)

# checking to ensure reshaping took appropriately
print("Dimensions of y after reshaping: {}".format(y.shape))
print("Dimensions of X after reshaping: {}".format(X.shape))

Dimensions of y after reshaping: (220320, 1)
Dimensions of X after reshaping: (11456640, 1)


In [ ]:
# step 1. split data into training and test sets

<div class="alert alert-success">
<H3> Step 4. Build the model </H3>
    
Sub-Steps involved:
    <ol>
        <li>___Parameter settings___: there are often a large number of parameters that can be adjusted here and with any modeling tool. Here I will list the parameters along with their chosen values, and my rationale for the choice of the parameter settings.</li>
        <li>___Models___: Here, I will state the actual models produced by the modeling tool.</li>
        <li>___Model descriptions___: Here, I will describe the resulting models, report on the interpretation of the models and document any difficulties encountered with their meanings.</li>
    </ol>

In [ ]:
# step 1. instantiate the regressor
# step 2. fit it on the training set
# step 3. predict on the test set
# step 4. score the model (score)
# step 5. score the model (Root Mean Squared Error (RMSE))

<div class="alert alert-success">
<H3> Step 5. Assess the model </H3>
    
Sub-Steps involved:
    <ol>
        <li>___Model assessment___: Here, I will summarize the results of this task, list the qualities of the generated models (e.g.in terms of accuracy) , and rank their performance to each other.</li>
        <li>___Revised parameter settings___: Here, I will state the actual models produced by the modeling tool.</li>
        <li>___Model descriptions___: Corresponding to the model assessment, here I will revise parameter settings and tune them for the next modeling run. Iterate model building and evaluation until I firmly believe that I have found the best model(s). Additionally, I will document any such revisions and assessments for reproducibility.</li>
    </ol>

In [ ]:
# STEP 1 Model Assessment

# the r-squared returned is dependent on the way I arbitrarily split the data up
# to ensure the model can generalise and or perform to unseen/new data it is important to deploy cross-validation (K-fold-CV)
# step 1 split data into five folds
# step 2 hold out first fold as a test set
# step 3 fit model against the remaining four folds
# step 4 predict on the test fold
# step 5 stagger holding the next fold as a test set and predicting on it until you are done



# Step 2 Revised Parameter Settings
# find a home for confusion matrix


# Model Descriptions